In [170]:
%load_ext autoreload
%autoreload 2
import spacy
import scispacy
from scispacy.linking import EntityLinker
import en_core_sci_lg
from scify.nlp import *
from scify.utils.GNBR_data import get_data_and_distributions

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## TODO
[x] fix entity shift in char offset

[x] merge entities yeah

[x] put utils in .py files. I hate those long Jupyter notebooks with their uncomposable trash python code all over the place 

[ ] 

[ ] can use multiple labels (since all are in entity._.kb_ents {bc5, jnlpa...}

[ ] Add more annotations as extension create product(UMLS, MESH, ontologies) X (models) annotated file from pubmed abstracts to not always load everything from scratch

[ ] slowly create multi-model NER annotated datasets to easy sample and do experiments on

[ ] look at source code for kindred, scipacy, role_pattern_nlp


UMLS is really big. Better to load linkers in separate cells. Sometimes crashes or similar on my Macbook Air

In [164]:
# This line takes a while, because we have to download ~1GB of data
# and load a large JSON file (the knowledge base). Be patient!
# Thankfully it should be faster after the first time you use it, because
# the downloads are cached.
# NOTE: The resolve_abbreviations parameter is optional, and requires that
# the AbbreviationDetector pipe has already been added to the pipeline. Adding
# the AbbreviationDetector pipe and setting resolve_abbreviations to True means
# that linking will only be performed on the long form of abbreviations.

#UMLS is big and takes a while EVEN when CACHED
linker_umls = EntityLinker(resolve_abbreviations=True, name="umls")

#linker_go = EntityLinker(resolve_abbreviations=True, name="go")
#linker_mesh = EntityLinker(resolve_abbreviations=True, name="mesh")

In [166]:
#linkers = [EntityLinker(resolve_abbreviations=True, name=ontology) for ontology in ["umls", "go", "mesh"]] this might crash
#linkers = [linker_mesh, linker_go, linker_umls]
#[get_ontology_name(linker) for linker in linkers]

#NER_models = [en_ner_bc5cdr_md, en_ner_craft_md, en_ner_bionlp13cg_md, en_ner_jnlpba_md]
#nlps = [model.load() for model in NER_models]

In [6]:
#Specialized NER models trained on specialized corpus
import en_ner_bc5cdr_md
import en_ner_craft_md
import en_ner_bionlp13cg_md
import en_ner_jnlpba_md

In [7]:
text="Spinal and bulbar muscular atrophy (SBMA) is an \
           inherited motor neuron disease caused by the expansion \
           of a polyglutamine tract within the androgen receptor (AR). \
           SBMA can be caused by this easily."

I'm loading models here in every cell (instead of mapping them) separately because it sometimes crashes...

In [8]:
base_nlp = spacy.load("en_core_sci_lg")

In [9]:
nlp1 = spacy.load("en_ner_bc5cdr_md")

In [10]:
nlp2 = spacy.load("en_ner_craft_md") #craft has loading issues? buggy....

In [11]:
nlp3 = en_ner_bionlp13cg_md.load()

In [12]:
nlp4 = en_ner_jnlpba_md.load()

In [13]:
NER_nlps = [nlp1, nlp2, nlp3, nlp4]

In [16]:
nlps = add_pipes_mutative(NER_nlps, linker_umls)

In [17]:
base_nlp.pipeline, nlps[3].pipeline, nlps[0].meta["name"]

([('tagger', <spacy.pipeline.pipes.Tagger at 0x20e9a6fd0>),
  ('parser', <spacy.pipeline.pipes.DependencyParser at 0x120580460>),
  ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x1905ec3a0>),
  ('AbbreviationDetector',
   <scispacy.abbreviation.AbbreviationDetector at 0x34ebcc6d0>),
  ('merge_entities', <function spacy.pipeline.functions.merge_entities(doc)>),
  ('EntityLinker', <scispacy.linking.EntityLinker at 0x109a181f0>)],
 [('tagger', <spacy.pipeline.pipes.Tagger at 0x302f6c9d0>),
  ('parser', <spacy.pipeline.pipes.DependencyParser at 0x31c4ecca0>),
  ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x20e4afa60>),
  ('AbbreviationDetector',
   <scispacy.abbreviation.AbbreviationDetector at 0x34ebccbe0>),
  ('merge_entities', <function spacy.pipeline.functions.merge_entities(doc)>),
  ('EntityLinker', <scispacy.linking.EntityLinker at 0x109a181f0>)])

Be Very careful to overwrite Ents .... there's many attributes that are needed! I just attach an extension
(but in the end, my algorithm just looks at labels_ to determine if a sentence is worthy of pattern matching!)

1) _token the product(NER modules x mesh/go/umls)
2) Read the Text with all of them
3) Extract their labels and ids + confidence scores
4) Look at set differences -> overlap intersect?
4) Add labels to doc @ Scispacy_lg (no NER) so it has Labels (set_extension)
5) Assign highest confidence label to scispacy large

In [146]:
#linker_umls.kb.cui_to_entity["5372017"]
#D016472 is a mesh ID

In [154]:
chem_gene_in_texts = [
    "The antiarrhythmic effect of carvedilol was examined in a placebo-controlled multicenter trial , the Carvedilol Post-Infarct Survival Control in Left_Ventricular_Dysfunction -LRB- CAPRICORN -RRB- study , which enrolled 1,959 patients with reduced left ventricular systolic function after AMI , 98 % of whom were treated with an ACE inhibitor .",
   "Carvedilol improves survival of patients suffering from CHF but the effects of the drug on angiotensin-converting_enzyme -LRB- ACE -RRB- activity , renin and aldosterone are not well characterized in patients receiving an ACE inhibitor .",
"We therefore assessed the effects of introducing carvedilol and bisoprolol in a prospective manner on indices of oxidative stress -LSB- lipid hydroperoxides -LRB- LHP -RRB- -RSB- , endothelial damage -LSB- von_Willebrand factor -LRB- vWf -RRB- -RSB- , platelet activation -LRB- soluble P-selectin -RRB- and coagulation -LRB- fibrinogen -RRB- and their inter-relationships in stable outpatients with CHF in sinus rhythm ."
]

In [169]:
docs = get_merged_docs_for_texts(chem_gene_in_texts, base_nlp, nlps[1:])

In [168]:
visualise_doc(docs[0], compact=True)

In [171]:
data, distributions = get_data_and_distributions()

['A+', 'A+.ind', 'A-', 'A-.ind', 'B', 'B.ind', 'E+', 'E+.ind', 'E-', 'E-.ind', 'E', 'E.ind', 'N', 'N.ind', 'O', 'O.ind', 'K', 'K.ind', 'Z', 'Z.ind'] headers 
        chemical-gene
(A+) agonism, activation
(A-) antagonism, blocking
(B) binding, ligand (esp. receptors)
(E+) increases expression/production
(E-) decreases expression/production
(E) affects expression/production (neutral)
(N) inhibits
        


In [ ]:
#get_non_zero support for one of the 30 themes only
#dependency paths + support
#Maybe merge dependency paths
#distributions

#it's very big but looks like this
distributions_kv_example = {'kinases|compound|start_entity participate|nsubj|kinases participate|nmod|end_entity': {'A+': 2.0,
  'A+.ind': 0.0,
  'A-': 0.0,
  'A-.ind': 0.0,
  'B': 0.0,
  'B.ind': 0.0,
  'E+': 0.0,
  'E+.ind': 0.0,
  'E-': 0.0,
  'E-.ind': 0.0,
  'E': 0.0,
  'E.ind': 0.0,
  'N': 0.0,
  'N.ind': 0.0,
  'O': 2.0,
  'O.ind': 0.0,
  'K': 0.0,
  'K.ind': 0.0,
  'Z': 0.0,
  'Z.ind': 0.0}